<a href="https://colab.research.google.com/github/ImagingDataCommons/TCIA-IDC-Coordination/blob/master/issue-27-prostatex/ProstateX_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ProstateX point annotations to DICOM SR TID1500

## Background

This notebook is WIP developed for the purposes of converting annotations of biopsy targets in the ProstateX TCIA collection into DICOM SR representation.

As is, this code is not final and may not work, but it is shared for the sake of transparency to facilitate similar conversion efforts.

Contact: andrey.fedorov AT gmail dot com

In [ ]:
%%bigquery prostatex_findings

select distinct(prostatex_fid_id), * from `idc-tcia.prostatex.findings_with_DICOM_UIDs`

Downloading: 100%|██████████| 544/544 [00:01<00:00, 506.44rows/s]


In [ ]:
from pathlib import Path

import numpy as np
import pydicom
from pydicom.uid import generate_uid
from pydicom.filereader import dcmread
from pydicom.sr.codedict import codes

from highdicom.sr.content import (
    FindingSite,
    ImageRegion,
    ImageRegion3D,
    SourceImageForRegion
)
from highdicom.sr.enum import GraphicTypeValues3D
from highdicom.sr.enum import GraphicTypeValues
from highdicom.sr.sop import Comprehensive3DSR, ComprehensiveSR
from highdicom.sr.templates import (
    DeviceObserverIdentifyingAttributes,
    Measurement,
    MeasurementProperties,
    MeasurementReport,
    ObservationContext,
    ObserverContext,
    PersonObserverIdentifyingAttributes,
    PlanarROIMeasurementsAndQualitativeEvaluations,
    RelationshipTypeValues,
    TrackingIdentifier,
)
from highdicom.sr.value_types import (
    CodedConcept,
    CodeContentItem,
)

import logging
logger = logging.getLogger("highdicom.sr.sop")
logger.setLevel(logging.INFO)

In [ ]:
# TODO: fix PixelOriginInterpretation to be FRAME - n/a, only for 2d
# TODO: use fid id as the tracking identifier - done
# TODO: what to do if no findings?
import re
def save_SR_for_case(findings_df, reference_dcm_file):

    # Path to single-frame CT image instance stored as PS3.10 file
    #image_file = Path('/Users/af61/Documents/TCIA/PROSTATEx/ProstateX-0000/07-07-2011-MR prostaat kanker detectie WDSmc MCAPRODETW-05711/7.000000-ep2ddifftraDYNDISTADC-48780/1-01.dcm')

    # Read CT Image data set from PS3.10 files on disk
    image_dataset = dcmread(reference_dcm_file)

    # Describe the context of reported observations: the person that reported
    # the observations and the device that was used to make the observations
    observer_person_context = ObserverContext(
        observer_type=codes.DCM.Person,
        observer_identifying_attributes=PersonObserverIdentifyingAttributes(
            name='Anonymous^Reader'
        )
    )
    observer_device_context = ObserverContext(
        observer_type=codes.DCM.Device,
        observer_identifying_attributes=DeviceObserverIdentifyingAttributes(
            uid=generate_uid()
        )
    )
    observation_context = ObservationContext(
        observer_person_context=observer_person_context,
        #observer_device_context=observer_device_context,
    )

    imaging_measurements = []

    for index,one_finding in findings_df.iterrows():

      print("Parsing "+one_finding['prostatex_pos'])

      finding_id = one_finding['prostatex_fid']

      pos_items = re.search(r"\ ?(-?[0-9]*\.[0-9]*)\ +(-?[0-9]*\.[0-9]*)\ +(-?[0-9]*\.[0-9]*)", one_finding['prostatex_pos'])
      if not pos_items:
        print("Failed to parse "+one_finding['prostatex_pos'])

      referenced_region_3d = ImageRegion3D(
          graphic_type=GraphicTypeValues3D.POINT,
          graphic_data=np.array([[float(pos_items.group(1)), float(pos_items.group(2)), float(pos_items.group(3))]]),
          #source_image=source_image_ref
          frame_of_reference_uid=one_finding['FrameOfReferenceUID']
      )

      # Describe the anatomic site at which observations were made
      if one_finding['prostatex_zone'] == 'TZ':
        finding_location = CodedConcept(
                      value="399384005",
                      meaning="Transition zone of the prostate",
                      scheme_designator="SCT"
                  )
      elif one_finding['prostatex_zone'] == 'PZ':
        finding_location = CodedConcept(
                      value="279706003",
                      meaning="Peripheral zone of the prostate",
                      scheme_designator="SCT"
                  )
      elif one_finding['prostatex_zone'] == 'AS':
        finding_location = CodedConcept(
                      value="717025007",
                      meaning="Anterior fibromuscular stroma of prostate",
                      scheme_designator="SCT"
                  )
      elif one_finding['prostatex_zone'] == 'SV':
        finding_location = CodedConcept(
                      value="64739004",
                      meaning="Seminal vesicle",
                      scheme_designator="SCT"
                  )
                     
      finding_sites = [
          FindingSite(anatomic_location=finding_location)
      ]


      evaluation = []
      ggg_code = None

      if one_finding['ClinSig'] == True:
        evaluation.append(CodeContentItem(CodedConcept(
                      value="RID49502",
                      meaning="clinically significant prostate cancer",
                      scheme_designator="RADLEX"
                  ), codes.SCT.Yes, RelationshipTypeValues.CONTAINS))
      elif one_finding['ClinSig'] == False:
        evaluation.append(CodeContentItem(CodedConcept(
                      value="RID49502",
                      meaning="clinically significant prostate cancer",
                      scheme_designator="RADLEX"
                  ), codes.SCT.No, RelationshipTypeValues.CONTAINS))
        
      if one_finding['ggg'] is not None:
        if one_finding['ggg'] == 1:
          ggg_code = CodedConcept(
                      value="860742008",
                      meaning="Gleason grade group 1",
                      scheme_designator="SCT"
                  )
        elif one_finding['ggg'] == 2:
          ggg_code = CodedConcept(
                      value="860743003",
                      meaning="Gleason grade group 2",
                      scheme_designator="SCT"
                  )
        elif one_finding['ggg'] == 3:
          ggg_code = CodedConcept(
                      value="860744009",
                      meaning="Gleason grade group 3",
                      scheme_designator="SCT"
                  )
        elif one_finding['ggg'] == 4:
          ggg_code = CodedConcept(
                      value="860745005",
                      meaning="Gleason grade group 4",
                      scheme_designator="SCT"
                  )
        elif one_finding['ggg'] == 5:
          ggg_code = CodedConcept(
                      value="860746006",
                      meaning="Gleason grade group 5",
                      scheme_designator="SCT"
                  )          

         
        if ggg_code is not None:
          evaluation.append(CodeContentItem(CodedConcept(
                      value="385377005",
                      meaning="Gleason grade finding for prostatic cancer",
                      scheme_designator="SCT"
                  ), 
                  ggg_code
                  , RelationshipTypeValues.CONTAINS))

                


      imaging_measurements.append(
          PlanarROIMeasurementsAndQualitativeEvaluations(
              tracking_identifier=TrackingIdentifier(
                  uid=generate_uid(),
                  identifier=str(finding_id)
              ),
              referenced_region=referenced_region_3d,
              finding_type=codes.SCT.Lesion,
              #measurements=measurements,
              qualitative_evaluations=evaluation,
              finding_sites=finding_sites
          )
      )

    print(len(imaging_measurements))

    # Create the report content
    procedure_code = CodedConcept(value="719178004", scheme_designator="SCT", 
                                  meaning="Multiparametric magnetic resonance imaging of prostate")
    measurement_report = MeasurementReport(
        observation_context=observation_context,
        procedure_reported=procedure_code,
        imaging_measurements=imaging_measurements
    )

    # Create the Structured Report instance
    series_instance_uid = generate_uid()
    sr_dataset = Comprehensive3DSR(
        evidence=[image_dataset],
        content=measurement_report[0],
        series_number=100,
        series_instance_uid=series_instance_uid,
        sop_instance_uid=generate_uid(),
        instance_number=1,
        manufacturer='IDC',
        is_complete = True,
        is_final=True
    )

    #print(sr_dataset)

    
    pydicom.write_file(image_dataset.PatientID+"_"+series_instance_uid+".dcm", sr_dataset)


import pandas as pd
from google.cloud import storage
import shutil
import os
import subprocess
from pydicom.filereader import dcmread

if not os.path.exists('prostatex'):
  os.makedirs('prostatex')

cases = prostatex_findings['prostatex_ProxID'].unique()

# skip those for now
# - first three reference multiple series and/or multiple FoRs
# - ProstateX-0005 has inconsistent clin_sig/ggg assignment
# - ProstateX-0331 is not present in prostatex
# - 
cases_to_skip = ['ProstateX-0025', 'ProstateX-0233', 'ProstateX-0223', 'ProstateX-0005', 'ProstateX-0331']

for case in cases:
  if case in cases_to_skip:
    continue
  findings_df = prostatex_findings[prostatex_findings["prostatex_ProxID"]==case]
  for gcs_url in findings_df['gcs_url']:
    local_file = os.path.join('prostatex',gcs_url.split('/')[-1])
    if not os.path.exists(local_file):
      gsutil_cmd = ['gsutil','-u','idc-tcia','cp',gcs_url,local_file]
      subprocess.run(gsutil_cmd)
      print(' '.join(gsutil_cmd))
  save_SR_for_case(findings_df, local_file)
  break

Parsing -0.78383 30.958 -29.0582
Parsing -4.64873 39.4624 -42.5734
2


In [ ]:
!dsrdump prostate_sr.dcm

Comprehensive 3D SR Document

Patient             : ProstateX-0207 (M, #ProstateX-0207)
Study               : MR prostaat kanker detectie_mc MCAPRODET
Manufacturer        : IDC
Preliminary Flag    : FINAL
Completion Flag     : COMPLETE
Verification Flag   : UNVERIFIED
Content Date/Time   : 2021-06-03 21:43:04

<CONTAINER:(,,"Imaging Measurement Report")=CONTINUOUS>
  <has concept mod CODE:(,,"Language of Content Item and Descendants")=(en-US,RFC5646,"English (United States)")>
  <has obs context CODE:(,,"Observer Type")=(121006,DCM,"Person")>
  <has obs context PNAME:(,,"Person Observer Name")="Anonymous^Reader">
  <has concept mod CODE:(,,"Procedure reported")=(719178004,SCT,"Multiparametric magnetic resonance imaging of prostate")>
  <contains CONTAINER:(,,"Image Library")=CONTINUOUS>
  <contains CONTAINER:(,,"Imaging Measurements")=CONTINUOUS>
    <contains CONTAINER:(,,"Measurement Group")=CONTINUOUS>
      <has obs context TEXT:(,,"Tracking Identifier")="2">
      <has obs context

In [ ]:
sr_dcm = pydicom.dcmread("ProstateX-0207_1.2.826.0.1.3680043.8.498.71275730577398178070714152792028903054.dcm")
sr_dcm.StudyInstanceUID

'1.3.6.1.4.1.14519.5.2.1.7310.5101.860473186348887719777907797922'

In [ ]:
%%bigquery study_instances

select gcs_url from `idc-dev-etl.idc_v2.dicom_all`
where StudyInstanceUID = '1.3.6.1.4.1.14519.5.2.1.7310.5101.860473186348887719777907797922'

Downloading: 100%|██████████| 922/922 [00:01<00:00, 802.81rows/s]


In [ ]:
manifest_name = sr_dcm.StudyInstanceUID+".txt"
with open(manifest_name,"w") as manifest_fp:
  for gcs_url in study_instances['gcs_url'].unique():
    manifest_fp.write(gcs_url+"\n")

In [ ]:
!mkdir 1.3.6.1.4.1.14519.5.2.1.7310.5101.860473186348887719777907797922
!cat 1.3.6.1.4.1.14519.5.2.1.7310.5101.860473186348887719777907797922.txt | gsutil -u idc-tcia -m cp -Ir gs://tcia-idc-datareviewcoordination/issue-27-prostatex

mkdir: cannot create directory ‘1.3.6.1.4.1.14519.5.2.1.7310.5101.860473186348887719777907797922’: File exists
Copying gs://idc_dev/53ca90ad-a990-4feb-8b54-d93c6d253b77.dcm [Content-Type=application/dicom]...
Copying gs://idc_dev/27aace1d-0187-4a04-839c-b103bd447e53.dcm [Content-Type=application/dicom]...
Copying gs://idc_dev/e53e75ba-deff-47b7-9a78-9d30c18dc04d.dcm [Content-Type=application/dicom]...
Copying gs://idc_dev/776046d2-8466-4b0f-9830-14cd81cfad0a.dcm [Content-Type=application/dicom]...
Copying gs://idc_dev/b6515941-d4b3-4217-82c9-896e058b634c.dcm [Content-Type=application/dicom]...
Copying gs://idc_dev/36a97972-bdac-46e3-b7f4-98823fccc6e4.dcm [Content-Type=application/dicom]...
Copying gs://idc_dev/074b3912-2e88-4d50-9b4a-218e50487b4f.dcm [Content-Type=application/dicom]...
Copying gs://idc_dev/81f2eeef-2170-43e8-a76d-cf7b9488953f.dcm [Content-Type=application/dicom]...
Copying gs://idc_dev/2480414d-7a5b-457d-ba18-f0007d73d63c.dcm [Content-Type=application/dicom]...
Copying

In [ ]:
!gsutil cp ProstateX-0207_1.2.826.0.1.3680043.8.498.71275730577398178070714152792028903054.dcm gs://tcia-idc-datareviewcoordination/issue-27-prostatex

Copying file://ProstateX-0207_1.2.826.0.1.3680043.8.498.71275730577398178070714152792028903054.dcm [Content-Type=application/dicom]...
/ [1 files][  6.0 KiB/  6.0 KiB]                                                
Operation completed over 1 objects/6.0 KiB.                                      


In [ ]:
!dcmdump ProstateX-0207_1.2.826.0.1.3680043.8.498.71275730577398178070714152792028903054.dcm | grep Modality

(0008,0060) CS [SR]                                     #   2, 1 Modality
